# Generate a metadata table with links to inference results

# Set-up

In [2]:
import os
import sys
import glob
import pandas as pd
import numpy as np

# GRNs

In [24]:
#dataset_name = "igvf_b01_LeftCortex"
dataset_name = "Bridge_Satpathy"
base_dir = "/cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference"
methods_run = ["aracne", "celloracle", "grnboost2", "scenic", "wgcna"]

In [25]:
run_metadata_paths = {}
metadata_df = pd.DataFrame()
for method in methods_run:
    csv_glob = os.path.join(base_dir, method, "bin", dataset_name, "successful_runs", "**", "*.csv")
    csv_files = glob.glob(csv_glob)
    run_metadata_paths[method] = csv_files
    for csv_file in csv_files:
        df = pd.read_csv(csv_file)
        df["method"] = method
        metadata_df = metadata_df.append(df)
metadata_df = metadata_df.reset_index(drop=True)
metadata_df.head()

/tmp/ipykernel_1870476/99281832.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metadata_df = metadata_df.append(df)
/tmp/ipykernel_1870476/99281832.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metadata_df = metadata_df.append(df)
/tmp/ipykernel_1870476/99281832.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metadata_df = metadata_df.append(df)
/tmp/ipykernel_1870476/99281832.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metadata_df = metadata_df.append(df)
/tmp/ipykernel_1870476/99281832.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.

,in_file_path,out_dir_path,tf_list,pval,num_boostraps,tf_type,run_id,method,base_grn,bagging_number,alpha,layer,num_runs,ranking_db,annotation_db,network_type
0,/cellar/users/aklie/data/igvf/topic_grn_links/...,/cellar/users/aklie/projects/igvf/topic_grn_li...,/cellar/users/aklie/opt/igvf-ucsd/aracne_pipel...,1.000000e-08,5.0,tf,2408877629_20230831-121420,aracne,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,/cellar/users/aklie/data/igvf/topic_grn_links/...,/cellar/users/aklie/projects/igvf/topic_grn_li...,/cellar/users/aklie/opt/igvf-ucsd/aracne_pipel...,1.000000e-08,5.0,tf,2408877629_20230831-121420,aracne,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,/cellar/users/aklie/data/igvf/topic_grn_links/...,/cellar/users/aklie/projects/igvf/topic_grn_li...,/cellar/users/aklie/opt/igvf-ucsd/aracne_pipel...,1.000000e-08,5.0,tf,2408877629_20230831-121420,aracne,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,/cellar/users/aklie/data/igvf/topic_grn_links/...,/cellar/users/aklie/projects/igvf/topic_grn_li...,/cellar/users/aklie/opt/igvf-ucsd/aracne_pipel...,1.000000e-08,5.0,tf,2408877629_20230831-121420,aracne,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,/cellar/users/aklie/data/igvf/topic_grn_links/...,/cellar/users/aklie/projects/igvf/topic_grn_li...,/cellar/users/aklie/opt/igvf-ucsd/aracne_pipel...,1.000000e-08,5.0,tf,2408877629_20230831-121420,aracne,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# For evert row in the dataframe, I want to check if there exists a file in the "out_dir_path" column called "net.tsv"
def check_net_tsv(row):
    out_dir_path = row["out_dir_path"]
    net_tsv_path = os.path.join(out_dir_path, "net.tsv")
    return os.path.exists(net_tsv_path)
metadata_df["net_tsv_exists"] = metadata_df.apply(check_net_tsv, axis=1)

In [27]:
metadata_df["method"].value_counts()

aracne        12
celloracle    12
grnboost2     12
scenic        12
wgcna         12
Name: method, dtype: int64

In [28]:
metadata_df[metadata_df["net_tsv_exists"] == True].method.value_counts()

aracne        12
celloracle    12
grnboost2     12
scenic        12
wgcna         12
Name: method, dtype: int64

In [29]:
metadata_df[metadata_df["net_tsv_exists"] == False].method.value_counts()

Series([], Name: method, dtype: int64)

# ARACNe clean-up

In [178]:
missing_aracne = metadata_df[(metadata_df["method"] == "aracne") & (metadata_df["net_tsv_exists"] == False)]
missing_aracne

,in_file_path,out_dir_path,tf_list,pval,num_boostraps,tf_type,run_id,method,base_grn,bagging_number,alpha,layer,num_runs,ranking_db,annotation_db,network_type,net_tsv_exists


In [25]:
#aracne_output_dirs = missing_aracne[missing_aracne["method"] == "aracne"]["out_dir_path"].values
aracne_output_dirs = metadata_df[metadata_df["method"] == "aracne"]["out_dir_path"].values

In [26]:
aracne_output_adapter_script = "/cellar/users/aklie/projects/igvf/topic_grn_links/opt/aracne_pipeline/bin/aracne_output_adapter.py"

In [27]:
# For every output dir run the python script
for output_dir in aracne_output_dirs:
    cmd = "python {} --aracne_out_dir {}".format(aracne_output_adapter_script, output_dir)
    print(cmd)
    os.system(cmd)

python /cellar/users/aklie/projects/igvf/topic_grn_links/opt/aracne_pipeline/bin/aracne_output_adapter.py --aracne_out_dir /cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/aracne/results/igvf_b01_LeftCortex/balanced_genotype_microglia/0.05/raw/B6J/tf


INFO:root:Adapter started.
INFO:root:Found 1 adjacencies files.
INFO:root:Found 21163 adjacencies in total.
INFO:root:Columns now: Index(['source', 'target', 'weight_unsigned', 'p', 'aracne_regulator_type'], dtype='object')
INFO:root:Columns now: Index(['source', 'target', 'weight_unsigned', 'p', 'aracne_regulator_type',
       'weight_minmax_normalized', 'description', '-logp', 'weight_signed'],
      dtype='object')
INFO:root:Process completed. Results saved at /cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/aracne/results/igvf_b01_LeftCortex/balanced_genotype_microglia/0.05/raw/B6J/tf/net.tsv. Duration: 2.68 seconds


python /cellar/users/aklie/projects/igvf/topic_grn_links/opt/aracne_pipeline/bin/aracne_output_adapter.py --aracne_out_dir /cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/aracne/results/igvf_b01_LeftCortex/balanced_genotype_microglia/0.05/raw/CASTJ/tf


INFO:root:Adapter started.
INFO:root:Found 1 adjacencies files.
INFO:root:Found 1860 adjacencies in total.
INFO:root:Columns now: Index(['source', 'target', 'weight_unsigned', 'p', 'aracne_regulator_type'], dtype='object')
INFO:root:Columns now: Index(['source', 'target', 'weight_unsigned', 'p', 'aracne_regulator_type',
       'weight_minmax_normalized', 'description', '-logp', 'weight_signed'],
      dtype='object')
INFO:root:Process completed. Results saved at /cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/aracne/results/igvf_b01_LeftCortex/balanced_genotype_microglia/0.05/raw/CASTJ/tf/net.tsv. Duration: 2.32 seconds


python /cellar/users/aklie/projects/igvf/topic_grn_links/opt/aracne_pipeline/bin/aracne_output_adapter.py --aracne_out_dir /cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/aracne/results/igvf_b01_LeftCortex/balanced_genotype_microglia/0.05/raw/both/tf


...

INFO:root:Adapter started.
INFO:root:Found 1 adjacencies files.
INFO:root:Found 41596 adjacencies in total.
INFO:root:Columns now: Index(['source', 'target', 'weight_unsigned', 'p', 'aracne_regulator_type'], dtype='object')
INFO:root:Columns now: Index(['source', 'target', 'weight_unsigned', 'p', 'aracne_regulator_type',
       'weight_minmax_normalized', 'description', '-logp', 'weight_signed'],
      dtype='object')
INFO:root:Process completed. Results saved at /cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/aracne/results/igvf_b01_LeftCortex/balanced_genotype_microglia/0.05/cpm_rank/B6J/tf/net.tsv. Duration: 2.82 seconds


python /cellar/users/aklie/projects/igvf/topic_grn_links/opt/aracne_pipeline/bin/aracne_output_adapter.py --aracne_out_dir /cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/aracne/results/igvf_b01_LeftCortex/balanced_genotype_microglia/0.05/cpm_rank/CASTJ/tf


INFO:root:Adapter started.
INFO:root:Found 1 adjacencies files.
INFO:root:Found 34302 adjacencies in total.
INFO:root:Columns now: Index(['source', 'target', 'weight_unsigned', 'p', 'aracne_regulator_type'], dtype='object')
INFO:root:Columns now: Index(['source', 'target', 'weight_unsigned', 'p', 'aracne_regulator_type',
       'weight_minmax_normalized', 'description', '-logp', 'weight_signed'],
      dtype='object')
INFO:root:Process completed. Results saved at /cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/aracne/results/igvf_b01_LeftCortex/balanced_genotype_microglia/0.05/cpm_rank/CASTJ/tf/net.tsv. Duration: 2.46 seconds


python /cellar/users/aklie/projects/igvf/topic_grn_links/opt/aracne_pipeline/bin/aracne_output_adapter.py --aracne_out_dir /cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/aracne/results/igvf_b01_LeftCortex/balanced_genotype_microglia/0.05/cpm_rank/both/tf


INFO:root:Adapter started.
INFO:root:Found 1 adjacencies files.
INFO:root:Found 49217 adjacencies in total.
INFO:root:Columns now: Index(['source', 'target', 'weight_unsigned', 'p', 'aracne_regulator_type'], dtype='object')
INFO:root:Columns now: Index(['source', 'target', 'weight_unsigned', 'p', 'aracne_regulator_type',
       'weight_minmax_normalized', 'description', '-logp', 'weight_signed'],
      dtype='object')
INFO:root:Process completed. Results saved at /cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/aracne/results/igvf_b01_LeftCortex/balanced_genotype_microglia/0.05/cpm_rank/both/tf/net.tsv. Duration: 3.24 seconds


# CellOracle clean-up

In [182]:
missing_celloracle = metadata_df[(metadata_df["method"] == "celloracle") & (metadata_df["net_tsv_exists"] == False)]
missing_celloracle

,in_file_path,out_dir_path,tf_list,pval,num_boostraps,tf_type,run_id,method,base_grn,bagging_number,alpha,layer,num_runs,ranking_db,annotation_db,network_type,net_tsv_exists


In [35]:
celloracle_output_dirs = metadata_df[metadata_df["method"] == "celloracle"]["out_dir_path"].values
celloracle_output_adapter_script = "/cellar/users/aklie/projects/igvf/topic_grn_links/opt/celloracle_pipeline/bin/grn_inference_only_pipeline/celloracle_output_adapter.py"

In [36]:
for output_dir in celloracle_output_dirs:
    cmd = "python {} --celloracle_out_dir {}".format(celloracle_output_adapter_script, output_dir)
    print(cmd)
    #os.system(cmd)

python /cellar/users/aklie/projects/igvf/topic_grn_links/opt/celloracle_pipeline/bin/grn_inference_only_pipeline/celloracle_output_adapter.py --celloracle_out_dir /cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/celloracle/results/Bridge_Satpathy/balanced_genotype_microglia/0.05/raw/B6J
python /cellar/users/aklie/projects/igvf/topic_grn_links/opt/celloracle_pipeline/bin/grn_inference_only_pipeline/celloracle_output_adapter.py --celloracle_out_dir /cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/celloracle/results/Bridge_Satpathy/balanced_genotype_microglia/0.05/raw/CASTJ
python /cellar/users/aklie/projects/igvf/topic_grn_links/opt/celloracle_pipeline/bin/grn_inference_only_pipeline/celloracle_output_adapter.py --celloracle_out_dir /cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/celloracle/results/Bridge_Satpathy/balanced_genotype_microglia/0.05/raw/both
python /cellar/users/aklie/projects/igvf/topic_grn_links/opt/celloracle_pipeline/bin/grn

In [40]:
os.system("python /cellar/users/aklie/projects/igvf/topic_grn_links/opt/celloracle_pipeline/bin/grn_inference_only_pipeline/celloracle_output_adapter.py --celloracle_out_dir /cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/celloracle/results/Bridge_Satpathy/balanced_genotype_microglia/0.05/raw/CASTJ")

INFO:root:Adapter started.
INFO:root:Process completed. Results saved at /cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/celloracle/results/Bridge_Satpathy/balanced_genotype_microglia/0.05/raw/CASTJ/net.tsv. Duration: 6.86 seconds


0

# GRNBoost2 clean-up

In [185]:
missing_grnboost2 = metadata_df[(metadata_df["method"] == "grnboost2") & (metadata_df["net_tsv_exists"] == False)]
missing_grnboost2

,in_file_path,out_dir_path,tf_list,pval,num_boostraps,tf_type,run_id,method,base_grn,bagging_number,alpha,layer,num_runs,ranking_db,annotation_db,network_type,net_tsv_exists
27,/cellar/users/aklie/data/igvf/topic_grn_links/...,/cellar/users/aklie/projects/igvf/topic_grn_li...,/cellar/users/aklie/opt/igvf-ucsd/genie3_pipel...,NaN,NaN,NaN,1256376291_20230830-120829,grnboost2,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,False
28,/cellar/users/aklie/data/igvf/topic_grn_links/...,/cellar/users/aklie/projects/igvf/topic_grn_li...,/cellar/users/aklie/opt/igvf-ucsd/genie3_pipel...,NaN,NaN,NaN,1256376291_20230830-120829,grnboost2,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,False
29,/cellar/users/aklie/data/igvf/topic_grn_links/...,/cellar/users/aklie/projects/igvf/topic_grn_li...,/cellar/users/aklie/opt/igvf-ucsd/genie3_pipel...,NaN,NaN,NaN,1256376291_20230830-120829,grnboost2,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,False
31,/cellar/users/aklie/data/igvf/topic_grn_links/...,/cellar/users/aklie/projects/igvf/topic_grn_li...,/cellar/users/aklie/opt/igvf-ucsd/grnboost2_pi...,NaN,NaN,NaN,140101058_20230831-122641,grnboost2,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,False
32,/cellar/users/aklie/data/igvf/topic_grn_links/...,/cellar/users/aklie/projects/igvf/topic_grn_li...,/cellar/users/aklie/opt/igvf-ucsd/grnboost2_pi...,NaN,NaN,NaN,140101058_20230831-122641,grnboost2,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,False
33,/cellar/users/aklie/data/igvf/topic_grn_links/...,/cellar/users/aklie/projects/igvf/topic_grn_li...,/cellar/users/aklie/opt/igvf-ucsd/grnboost2_pi...,NaN,NaN,NaN,140101058_20230831-122641,grnboost2,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,False


In [186]:
grnboost2_output_dirs = missing_grnboost2[missing_grnboost2["method"] == "grnboost2"]["out_dir_path"].values
grnboost2_loom_files = missing_grnboost2[missing_grnboost2["method"] == "grnboost2"]["in_file_path"].values
grnboost2_output_adapter_script = "/cellar/users/aklie/projects/igvf/topic_grn_links/opt/grnboost2_pipeline/bin/grnboost2_output_adapter.py"

In [187]:
# For every output dir run the python script
for i in range(len(grnboost2_output_dirs)):
    cmd = "python {} --grnboost2_out_dir {} --loom_file {}".format(grnboost2_output_adapter_script, grnboost2_output_dirs[i], grnboost2_loom_files[i])
    print(cmd)
    #os.system(cmd)

python /cellar/users/aklie/projects/igvf/topic_grn_links/opt/grnboost2_pipeline/bin/grnboost2_output_adapter.py --grnboost2_out_dir /cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/grnboost2/results/Bridge_Satpathy/balanced_genotype_microglia/0.05/pf_log1p_pf/B6J --loom_file /cellar/users/aklie/data/igvf/topic_grn_links/subsets/for_regulon_inference/Bridge_Satpathy/29Aug23/3744376661_20230829-175127/balanced_genotype_microglia/0.05/pf_log1p_pf/balanced_genotype_microglia-B6J.0.05.pf_log1p_pf.loom
python /cellar/users/aklie/projects/igvf/topic_grn_links/opt/grnboost2_pipeline/bin/grnboost2_output_adapter.py --grnboost2_out_dir /cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/grnboost2/results/Bridge_Satpathy/balanced_genotype_microglia/0.05/pf_log1p_pf/CASTJ --loom_file /cellar/users/aklie/data/igvf/topic_grn_links/subsets/for_regulon_inference/Bridge_Satpathy/29Aug23/3744376661_20230829-175127/balanced_genotype_microglia/0.05/pf_log1p_pf/balanced_genotype_

# SCENIC clean-up

In [15]:
missing_scenic = metadata_df[(metadata_df["method"] == "scenic") & (metadata_df["net_tsv_exists"] == False)]
missing_scenic

,in_file_path,out_dir_path,tf_list,pval,num_boostraps,tf_type,run_id,method,base_grn,bagging_number,alpha,layer,num_runs,ranking_db,annotation_db,network_type,net_tsv_exists
43,/cellar/users/aklie/data/igvf/topic_grn_links/...,/cellar/users/aklie/projects/igvf/topic_grn_li...,/cellar/users/aklie/opt/igvf-ucsd/genie3_pipel...,NaN,NaN,NaN,398626224_20230901-120046,scenic,NaN,NaN,NaN,NaN,5.0,/cellar/users/aklie/opt/igvf-ucsd/scenic_pipel...,/cellar/users/aklie/opt/igvf-ucsd/scenic_pipel...,NaN,False


In [30]:
scenic_output_dirs = metadata_df[metadata_df["method"] == "scenic"]["out_dir_path"].values
scenic_loom_files = metadata_df[metadata_df["method"] == "scenic"]["in_file_path"].values
scenic_output_adapter_script = "/cellar/users/aklie/projects/igvf/topic_grn_links/opt/scenic_pipeline/bin/scenic_output_adapter.py"
grnboost2_output_adapter_script = "/cellar/users/aklie/projects/igvf/topic_grn_links/opt/grnboost2_pipeline/bin/grnboost2_output_adapter.py"

In [33]:
# For every output dir run the python script
for output_dir in scenic_output_dirs:
    cmd = "python {} --grnboost2_out_dir {} --out_file grnboost2.tsv".format(grnboost2_output_adapter_script, output_dir)
    print(cmd)
    #os.system(cmd)

python /cellar/users/aklie/projects/igvf/topic_grn_links/opt/grnboost2_pipeline/bin/grnboost2_output_adapter.py --grnboost2_out_dir /cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/scenic/results/Bridge_Satpathy/balanced_genotype_microglia/0.05/log1p_cp10k/B6J --out_file grnboost2.tsv
python /cellar/users/aklie/projects/igvf/topic_grn_links/opt/grnboost2_pipeline/bin/grnboost2_output_adapter.py --grnboost2_out_dir /cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/scenic/results/Bridge_Satpathy/balanced_genotype_microglia/0.05/log1p_cp10k/CASTJ --out_file grnboost2.tsv
python /cellar/users/aklie/projects/igvf/topic_grn_links/opt/grnboost2_pipeline/bin/grnboost2_output_adapter.py --grnboost2_out_dir /cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/scenic/results/Bridge_Satpathy/balanced_genotype_microglia/0.05/log1p_cp10k/both --out_file grnboost2.tsv
python /cellar/users/aklie/projects/igvf/topic_grn_links/opt/grnboost2_pipeline/bin/grnboost2_

In [32]:
# For every output dir run the python script
for i in range(len(scenic_output_dirs)):
    cmd = "python {} --scenic_out_dir {} --loom_file {}".format(scenic_output_adapter_script, scenic_output_dirs[i], scenic_loom_files[i])
    print(cmd)
    #os.system(cmd)

python /cellar/users/aklie/projects/igvf/topic_grn_links/opt/scenic_pipeline/bin/scenic_output_adapter.py --scenic_out_dir /cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/scenic/results/Bridge_Satpathy/balanced_genotype_microglia/0.05/log1p_cp10k/B6J --loom_file /cellar/users/aklie/data/igvf/topic_grn_links/subsets/for_regulon_inference/Bridge_Satpathy/29Aug23/3744376661_20230829-175127/balanced_genotype_microglia/0.05/log1p_cp10k/balanced_genotype_microglia-B6J.0.05.log1p_cp10k.loom
python /cellar/users/aklie/projects/igvf/topic_grn_links/opt/scenic_pipeline/bin/scenic_output_adapter.py --scenic_out_dir /cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/scenic/results/Bridge_Satpathy/balanced_genotype_microglia/0.05/log1p_cp10k/CASTJ --loom_file /cellar/users/aklie/data/igvf/topic_grn_links/subsets/for_regulon_inference/Bridge_Satpathy/29Aug23/3744376661_20230829-175127/balanced_genotype_microglia/0.05/log1p_cp10k/balanced_genotype_microglia-CASTJ.0.05.log

# WGCNA clean-up

In [28]:
wgcna_output_adapter_script = "/cellar/users/aklie/projects/igvf/topic_grn_links/opt/wgcna_pipeline/bin/pywgcna_grn_inference_pipeline/wgcna_output_adapter.py"

In [15]:
missing_wgcna = metadata_df[(metadata_df["method"] == "wgcna") & (metadata_df["net_tsv_exists"] == False)]
missing_wgcna

,in_file_path,out_dir_path,tf_list,pval,num_boostraps,tf_type,run_id,method,base_grn,bagging_number,alpha,layer,num_runs,ranking_db,annotation_db,network_type,net_tsv_exists


In [29]:
#wgcna_output_dirs = missing_wgcna[missing_wgcna["method"] == "wgcna"]["out_dir_path"].values
wgcna_output_dirs = metadata_df[metadata_df["method"] == "wgcna"]["out_dir_path"].values

In [30]:
# For every output dir run the python script
for output_dir in wgcna_output_dirs:
    cmd = "python {} --wgcna_out_dir {}".format(wgcna_output_adapter_script, output_dir)
    print(cmd)
    os.system(cmd)

python /cellar/users/aklie/projects/igvf/topic_grn_links/opt/wgcna_pipeline/bin/pywgcna_grn_inference_pipeline/wgcna_output_adapter.py --wgcna_out_dir /cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/wgcna/results/igvf_b01_LeftCortex/balanced_genotype_microglia/0.05/raw/B6J


INFO:root:Adapter started.
INFO:root:Process completed. Results saved at /cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/wgcna/results/igvf_b01_LeftCortex/balanced_genotype_microglia/0.05/raw/B6J. Duration: 19.70 seconds


python /cellar/users/aklie/projects/igvf/topic_grn_links/opt/wgcna_pipeline/bin/pywgcna_grn_inference_pipeline/wgcna_output_adapter.py --wgcna_out_dir /cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/wgcna/results/igvf_b01_LeftCortex/balanced_genotype_microglia/0.05/raw/CASTJ


INFO:root:Adapter started.
INFO:root:Process completed. Results saved at /cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/wgcna/results/igvf_b01_LeftCortex/balanced_genotype_microglia/0.05/raw/CASTJ. Duration: 19.76 seconds


python /cellar/users/aklie/projects/igvf/topic_grn_links/opt/wgcna_pipeline/bin/pywgcna_grn_inference_pipeline/wgcna_output_adapter.py --wgcna_out_dir /cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/wgcna/results/igvf_b01_LeftCortex/balanced_genotype_microglia/0.05/raw/both


...

INFO:root:Adapter started.
INFO:root:Process completed. Results saved at /cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/wgcna/results/igvf_b01_LeftCortex/balanced_genotype_microglia/0.05/cpm_rank/B6J. Duration: 19.05 seconds


python /cellar/users/aklie/projects/igvf/topic_grn_links/opt/wgcna_pipeline/bin/pywgcna_grn_inference_pipeline/wgcna_output_adapter.py --wgcna_out_dir /cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/wgcna/results/igvf_b01_LeftCortex/balanced_genotype_microglia/0.05/cpm_rank/CASTJ


INFO:root:Adapter started.
INFO:root:Process completed. Results saved at /cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/wgcna/results/igvf_b01_LeftCortex/balanced_genotype_microglia/0.05/cpm_rank/CASTJ. Duration: 18.24 seconds


python /cellar/users/aklie/projects/igvf/topic_grn_links/opt/wgcna_pipeline/bin/pywgcna_grn_inference_pipeline/wgcna_output_adapter.py --wgcna_out_dir /cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/wgcna/results/igvf_b01_LeftCortex/balanced_genotype_microglia/0.05/cpm_rank/both


INFO:root:Adapter started.
INFO:root:Process completed. Results saved at /cellar/users/aklie/projects/igvf/topic_grn_links/grn_inference/wgcna/results/igvf_b01_LeftCortex/balanced_genotype_microglia/0.05/cpm_rank/both. Duration: 20.11 seconds


# Create a metadata table with all these tsvs

In [194]:
metadata_df[["cells", "genes", "normalization"]] = [i.split("/")[-4:-1] for i in metadata_df["in_file_path"]]
metadata_df["genotype"] = [i.split("/")[-1].split(".")[0].split("-")[-1] for i in metadata_df["in_file_path"]]
metadata_df

,in_file_path,out_dir_path,tf_list,pval,num_boostraps,tf_type,run_id,method,base_grn,bagging_number,alpha,layer,num_runs,network_type,net_tsv_exists,cells,genes,normalization,genotype
0,/cellar/users/aklie/data/igvf/topic_grn_links/...,/cellar/users/aklie/projects/igvf/topic_grn_li...,/cellar/users/aklie/opt/igvf-ucsd/aracne_pipel...,1.000000e-08,5.0,tf,3148063594_20230831-152712,aracne,NaN,NaN,NaN,NaN,NaN,NaN,True,balanced_genotype_microglia,0.05,raw,B6J
1,/cellar/users/aklie/data/igvf/topic_grn_links/...,/cellar/users/aklie/projects/igvf/topic_grn_li...,/cellar/users/aklie/opt/igvf-ucsd/aracne_pipel...,1.000000e-08,5.0,tf,3148063594_20230831-152712,aracne,NaN,NaN,NaN,NaN,NaN,NaN,True,balanced_genotype_microglia,0.05,raw,CASTJ
2,/cellar/users/aklie/data/igvf/topic_grn_links/...,/cellar/users/aklie/projects/igvf/topic_grn_li...,/cellar/users/aklie/opt/igvf-ucsd/aracne_pipel...,1.000000e-08,5.0,tf,3148063594_20230831-152712,aracne,NaN,NaN,NaN,NaN,NaN,NaN,True,balanced_genotype_microglia,0.05,raw,both
3,/cellar/users/aklie/data/igvf/topic_grn_links/...,/cellar/users/aklie/projects/igvf/topic_grn_li...,/cellar/users/aklie/opt/igvf-ucsd/aracne_pipel...,1.000000e-08,5.0,tf,3148063594_20230831-152712,aracne,NaN,NaN,NaN,NaN,NaN,NaN,True,balanced_genotype_microglia,0.05,log1p_cp10k,B6J
4,/cellar/users/aklie/data/igvf/topic_grn_links/...,/cellar/users/aklie/projects/igvf/topic_grn_li...,/cellar/users/aklie/opt/igvf-ucsd/aracne_pipel...,1.000000e-08,5.0,tf,3148063594_20230831-152712,aracne,NaN,NaN,NaN,NaN,NaN,NaN,True,balanced_genotype_microglia,0.05,log1p_cp10k,CASTJ
5,/cellar/users/aklie/data/igvf/topic_grn_links/...,/cellar/users/aklie/projects/igvf/topic_grn_li...,/cellar/users/aklie/opt/igvf-ucsd/aracne_pipel...,1.000000e-08,5.0,tf,3148063594_20230831-152712,aracne,NaN,NaN,NaN,NaN,NaN,NaN,True,balanced_genotype_microglia,0.05,log1p_cp10k,both
6,/cellar/users/aklie/data/igvf/topic_grn_links/...,/cellar/users/aklie/projects/igvf/topic_grn_li...,/cellar/users/aklie/opt/igvf-ucsd/aracne_pipel...,1.000000e-08,5.0,tf,3148063594_20230831-152712,aracne,NaN,NaN,NaN,NaN,NaN,NaN,True,balanced_genotype_microglia,0.05,pf_log1p_pf,B6J
7,/cellar/users/aklie/data/igvf/topic_grn_links/...,/cellar/users/aklie/projects/igvf/topic_grn_li...,/cellar/users/aklie/opt/igvf-ucsd/aracne_pipel...,1.000000e-08,5.0,tf,3148063594_20230831-152712,aracne,NaN,NaN,NaN,NaN,NaN,NaN,True,balanced_genotype_microglia,0.05,pf_log1p_pf,CASTJ
8,/cellar/users/aklie/data/igvf/topic_grn_links/...,/cellar/users/aklie/projects/igvf/topic_grn_li...,/cellar/users/aklie/opt/igvf-ucsd/aracne_pipel...,1.000000e-08,5.0,tf,3148063594_20230831-152712,aracne,NaN,NaN,NaN,NaN,NaN,NaN,True,balanced_genotype_microglia,0.05,pf_log1p_pf,both
9,/cellar/users/aklie/data/igvf/topic_grn_links/...,/cellar/users/aklie/projects/igvf/topic_grn_li...,/cellar/users/aklie/opt/igvf-ucsd/aracne_pipel...,1.000000e-08,5.0,tf,3148063594_20230831-152712,aracne,NaN,NaN,NaN,NaN,NaN,NaN,True,balanced_genotype_microglia,0.05,cpm_rank,B6J


In [195]:
metadata_df["output_table_path"] = metadata_df["out_dir_path"].apply(lambda x: os.path.join(x, "net.tsv"))
metadata_df["output_table_exists"] = metadata_df["output_table_path"].apply(lambda x: os.path.exists(x))
metadata_df["output_table_exists"].value_counts()

True    48
Name: output_table_exists, dtype: int64

In [200]:
metadata_out_dir = f"/cellar/users/aklie/projects/igvf/topic_grn_links/eval/metadata/{dataset_name}/01Sep23"
os.makedirs(metadata_out_dir, exist_ok=True)

In [201]:
os.chdir(f"/cellar/users/aklie/projects/igvf/topic_grn_links/eval/metadata/{dataset_name}")
os.getcwd()

'/cellar/users/aklie/projects/igvf/topic_grn_links/eval/metadata/igvf_b01_LeftCortex'

In [202]:
metadata_df.to_csv("01Sep23/eval_metadata.tsv", index=False, sep="\t")

In [ ]:
len(metadata_df)

39

# Scratch

## Back when we had heterogeneous outputs

In [ ]:
result_tbls = []
for i, row in metadata_df.iterrows():
    path = row["out_dir_path"] + "/network.txt"
    if os.path.exists(path):
        tables.append(path)
    else:
        tables.append(np.nan)


In [39]:
inference_tables = []
for i, row in metadata_df.iterrows():
    if row["method"] == "aracne":
        path = row["out_dir_path"] + "/network.txt"
        if os.path.exists(path):
            inference_tables.append(path)
        else:
            inference_tables.append(np.nan)
    elif row["method"] == "celloracle":
        path = row["out_dir_path"] + "/inference_result.tsv"
        if os.path.exists(path):
            inference_tables.append(path)
        else:
            inference_tables.append(np.nan)
    elif row["method"] == "genie3":
        path = row["out_dir_path"] + "/run1_adj.tsv"
        if os.path.exists(path):
            inference_tables.append(path)
        else:
            inference_tables.append(np.nan)
    elif row["method"] == "grnboost2":
        path = row["out_dir_path"] + "/run1_adj.tsv"
        if os.path.exists(path):
            inference_tables.append(path)
        else:
            inference_tables.append(np.nan)
    elif row["method"] == "scenic":
        path = row["out_dir_path"] + "/run1_reg.csv"
        if os.path.exists(path):
            inference_tables.append(path)
        else:
            inference_tables.append(np.nan)
    elif row["method"] == "wgcna":
        path = row["out_dir_path"] + "/corr_adj_list.tsv"
        if os.path.exists(path):
            inference_tables.append(path)
        else:
            inference_tables.append(np.nan)
    else:
        print("Method not found")
metadata_df["inference_table"] = inference_tables

In [40]:
metadata_df

,in_file_path,out_dir_path,tf_list,pval,num_boostraps,tf_type,run_id,method,base_grn,bagging_number,...,layer,num_runs,ranking_db,annotation_db,network_type,cells,genes,normalization,inference_table,genotype
0,/cellar/users/aklie/data/igvf/topic_grn_links/...,/cellar/users/aklie/projects/igvf/topic_grn_li...,/cellar/users/aklie/opt/igvf-ucsd/aracne_pipel...,1.000000e-08,5.0,tf,2739668979_20230830-115831,aracne,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,balanced_genotype_microglia,0.05,log1p_cp10k,/cellar/users/aklie/projects/igvf/topic_grn_li...,B6J
1,/cellar/users/aklie/data/igvf/topic_grn_links/...,/cellar/users/aklie/projects/igvf/topic_grn_li...,/cellar/users/aklie/opt/igvf-ucsd/aracne_pipel...,1.000000e-08,5.0,tf,2739668979_20230830-115831,aracne,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,balanced_genotype_microglia,0.05,log1p_cp10k,/cellar/users/aklie/projects/igvf/topic_grn_li...,CASTJ
2,/cellar/users/aklie/data/igvf/topic_grn_links/...,/cellar/users/aklie/projects/igvf/topic_grn_li...,/cellar/users/aklie/opt/igvf-ucsd/aracne_pipel...,1.000000e-08,5.0,tf,2739668979_20230830-115831,aracne,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,balanced_genotype_microglia,0.05,log1p_cp10k,/cellar/users/aklie/projects/igvf/topic_grn_li...,both
3,/cellar/users/aklie/data/igvf/topic_grn_links/...,/cellar/users/aklie/projects/igvf/topic_grn_li...,/cellar/users/aklie/opt/igvf-ucsd/aracne_pipel...,1.000000e-08,5.0,tf,2739668979_20230830-115831,aracne,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,balanced_genotype_microglia,0.05,cpm_rank,/cellar/users/aklie/projects/igvf/topic_grn_li...,B6J
4,/cellar/users/aklie/data/igvf/topic_grn_links/...,/cellar/users/aklie/projects/igvf/topic_grn_li...,/cellar/users/aklie/opt/igvf-ucsd/aracne_pipel...,1.000000e-08,5.0,tf,2739668979_20230830-115831,aracne,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,balanced_genotype_microglia,0.05,cpm_rank,/cellar/users/aklie/projects/igvf/topic_grn_li...,CASTJ
5,/cellar/users/aklie/data/igvf/topic_grn_links/...,/cellar/users/aklie/projects/igvf/topic_grn_li...,/cellar/users/aklie/opt/igvf-ucsd/aracne_pipel...,1.000000e-08,5.0,tf,2739668979_20230830-115831,aracne,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,balanced_genotype_microglia,0.05,cpm_rank,/cellar/users/aklie/projects/igvf/topic_grn_li...,both
6,/cellar/users/aklie/data/igvf/topic_grn_links/...,/cellar/users/aklie/projects/igvf/topic_grn_li...,NaN,NaN,NaN,NaN,3503408525_20230830-164904,celloracle,mouse_scATAC_atlas,20.0,...,normalized_counts,NaN,NaN,NaN,NaN,balanced_genotype_microglia,0.05,raw,/cellar/users/aklie/projects/igvf/topic_grn_li...,B6J
7,/cellar/users/aklie/data/igvf/topic_grn_links/...,/cellar/users/aklie/projects/igvf/topic_grn_li...,NaN,NaN,NaN,NaN,3503408525_20230830-164904,celloracle,mouse_scATAC_atlas,20.0,...,normalized_counts,NaN,NaN,NaN,NaN,balanced_genotype_microglia,0.05,raw,/cellar/users/aklie/projects/igvf/topic_grn_li...,CASTJ
8,/cellar/users/aklie/data/igvf/topic_grn_links/...,/cellar/users/aklie/projects/igvf/topic_grn_li...,NaN,NaN,NaN,NaN,3503408525_20230830-164904,celloracle,mouse_scATAC_atlas,20.0,...,normalized_counts,NaN,NaN,NaN,NaN,balanced_genotype_microglia,0.05,raw,/cellar/users/aklie/projects/igvf/topic_grn_li...,both
9,/cellar/users/aklie/data/igvf/topic_grn_links/...,/cellar/users/aklie/projects/igvf/topic_grn_li...,NaN,NaN,NaN,NaN,3503408525_20230830-164904,celloracle,mouse_scATAC_atlas,20.0,...,normalized_counts,NaN,NaN,NaN,NaN,balanced_genotype_microglia,0.05,log1p_cp10k,/cellar/users/aklie/projects/igvf/topic_grn_li...,B6J


In [41]:
os.chdir("/cellar/users/aklie/projects/igvf/topic_grn_links/eval/metadata/Bridge_Satpathy")
os.getcwd()

'/cellar/users/aklie/projects/igvf/topic_grn_links/eval/metadata/Bridge_Satpathy'

In [42]:
metadata_df.to_csv("31Aug23/eval_metadata.tsv", index=False, sep="\t")

In [43]:
len(metadata_df)

39

# DONE!

---